In [87]:
%pip install boto3

  Obtaining dependency information for boto3 from https://files.pythonhosted.org/packages/a1/b5/398d27383439318c33af4818646613032c48056e852af5b560b45ee0ba80/boto3-1.34.112-py3-none-any.whl.metadata
  Obtaining dependency information for botocore<1.35.0,>=1.34.112 from https://files.pythonhosted.org/packages/85/d5/fd0d3e2b6b2b488260966538497f12387d1407046428261ace8ac694a691/botocore-1.34.112-py3-none-any.whl.metadata
  Obtaining dependency information for s3transfer<0.11.0,>=0.10.0 from https://files.pythonhosted.org/packages/83/37/395cdb6ee92925fa211e55d8f07b9f93cf93f60d7d4ce5e66fd73f1ea986/s3transfer-0.10.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 24.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.76
    Uninstalling botocore-1.

In [2]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import teamgamelog
from nba_api.stats.endpoints import commonteamroster
import pandas as pd
import os

In [2]:
nba_teams = teams.get_teams()
nba_teams

[{'id': 1610612737,
  'full_name': 'Atlanta Hawks',
  'abbreviation': 'ATL',
  'nickname': 'Hawks',
  'city': 'Atlanta',
  'state': 'Georgia',
  'year_founded': 1949},
 {'id': 1610612738,
  'full_name': 'Boston Celtics',
  'abbreviation': 'BOS',
  'nickname': 'Celtics',
  'city': 'Boston',
  'state': 'Massachusetts',
  'year_founded': 1946},
 {'id': 1610612739,
  'full_name': 'Cleveland Cavaliers',
  'abbreviation': 'CLE',
  'nickname': 'Cavaliers',
  'city': 'Cleveland',
  'state': 'Ohio',
  'year_founded': 1970},
 {'id': 1610612740,
  'full_name': 'New Orleans Pelicans',
  'abbreviation': 'NOP',
  'nickname': 'Pelicans',
  'city': 'New Orleans',
  'state': 'Louisiana',
  'year_founded': 2002},
 {'id': 1610612741,
  'full_name': 'Chicago Bulls',
  'abbreviation': 'CHI',
  'nickname': 'Bulls',
  'city': 'Chicago',
  'state': 'Illinois',
  'year_founded': 1966},
 {'id': 1610612742,
  'full_name': 'Dallas Mavericks',
  'abbreviation': 'DAL',
  'nickname': 'Mavericks',
  'city': 'Dallas',

In [54]:
from nba_api.stats.endpoints import scoreboardv2
import pandas as pd

# Define the date for which you want to fetch match stats (format: 'YYYYMMDD')
game_date = '2024-03-29'
league_id = '00'

# Fetch the scoreboard for the specified date
scoreboard = scoreboardv2.ScoreboardV2(game_date=game_date, league_id=league_id)

In [76]:
# Get the game IDs from the response
game_header = scoreboard.game_header.get_data_frame()
game_ids = game_header['GAME_ID'].tolist()

In [80]:
from nba_api.stats.endpoints import boxscoretraditionalv2

# Create lists to hold team and player stats
all_team_stats = []
all_player_stats = []

# Loop through each game ID to fetch detailed stats
for game_id in game_ids:
    try:
        # Fetch box score for the game
        boxscore = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=game_id)
        
        # Get team and player stats data frames
        team_stats_df = boxscore.team_stats.get_data_frame()
        player_stats_df = boxscore.player_stats.get_data_frame()
        
        # Add game ID to the data frames
        team_stats_df['GAME_ID'] = game_id
        player_stats_df['GAME_ID'] = game_id
        
        # Append the stats to the lists
        all_team_stats.append(team_stats_df)
        all_player_stats.append(player_stats_df)
    except Exception as e:
        print(f"An error occurred for game ID {game_id}: {e}")

# Combine all team and player stats into single data frames
all_team_stats_df = pd.concat(all_team_stats, ignore_index=True)
all_player_stats_df = pd.concat(all_player_stats, ignore_index=True)

# Write the combined DataFrame to a CSV file
all_player_stats_df.to_csv('nba_player_stats_2024_04_29.csv', index=False)
all_team_stats_df.to_csv('nba_team_stats_2024_04_29.csv', index=False)

In [5]:
# Fetch all NBA teams
nba_teams = teams.get_teams()

# Convert the list of teams to a DataFrame
teams_df = pd.DataFrame(nba_teams)

for row in 

# Loop through each team to fetch coach details
for index, row in teams_df.iterrows():
    team_id = row['id']
    try:
        # Fetch team roster and details for the given team ID
        team_roster = commonteamroster.CommonTeamRoster(team_id=team_id)
        coaches_df = team_roster.coaches.get_data_frame()  # The coaches dataframe
        
        # Filter for head coach (assuming 'Head Coach' in COACH_TYPE)
        head_coach = coaches_df[coaches_df['COACH_TYPE'] == 'Head Coach'].iloc[0]
        
        # Extract coach details
        coach_name = head_coach['COACH_NAME']
        is_assistant = head_coach['IS_ASSISTANT']
        
        # Add the details to the DataFrame
        teams_df.at[index, 'coach_name'] = coach_name
        teams_df.at[index, 'is_assistant'] = is_assistant
    except Exception as e:
        print(f"An error occurred for team ID {team_id}: {e}")

# Display the enriched team details
print("Enriched NBA Team Details with Coaches:")
print(teams_df.head())

# Save the enriched team details to a CSV file
teams_df.to_csv('nba_teams_coaches_details.csv', index=False)
print("Enriched team details with coaches have been written to nba_teams_coaches_details.csv")


An error occurred for team ID 1610612747: single positional indexer is out-of-bounds
An error occurred for team ID 1610612756: single positional indexer is out-of-bounds
An error occurred for team ID 1610612764: single positional indexer is out-of-bounds
An error occurred for team ID 1610612765: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
An error occurred for team ID 1610612766: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Enriched NBA Team Details with Coaches:
           id             full_name abbreviation   nickname         city  \
0  1610612737         Atlanta Hawks          ATL      Hawks      Atlanta   
1  1610612738        Boston Celtics          BOS    Celtics       Boston   
2  1610612739   Cleveland Cavaliers          CLE  Cavaliers    Cleveland   
3  1610612740  New Orleans Pelicans          NOP   Pelicans  New Orleans   
4  1610612741         Chicago Bulls          CHI      B

In [2]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import teaminfocommon
import pandas as pd

# Fetch all NBA teams
nba_teams = teams.get_teams()

# Convert the list of teams to a DataFrame
teams_df = pd.DataFrame(nba_teams)

# Add columns for conference and division
teams_df['conference'] = None
teams_df['division'] = None

# Loop through each team to fetch conference and division details
for index, row in teams_df.iterrows():
    team_id = row['id']
    try:
        # Fetch team details for the given team ID
        team_info_common = teaminfocommon.TeamInfoCommon(team_id=team_id)
        team_info = team_info_common.team_info_common.get_data_frame().iloc[0]
        
        # Extract conference and division information
        conference = team_info['TEAM_CONFERENCE']
        division = team_info['TEAM_DIVISION']
        
        # Add the details to the DataFrame
        teams_df.at[index, 'conference'] = conference
        teams_df.at[index, 'division'] = division
    except Exception as e:
        print(f"An error occurred for team ID {team_id}: {e}")

# Display the enriched team details
print("Enriched NBA Team Details with Conference and Division:")
print(teams_df.head())

# Save the enriched team details to a CSV file
teams_df.to_csv('nba_teams_conference_division.csv', index=False)
print("Enriched team details with conference and division have been written to nba_teams_conference_division.csv")

Enriched NBA Team Details with Conference and Division:
           id             full_name abbreviation   nickname         city  \
0  1610612737         Atlanta Hawks          ATL      Hawks      Atlanta   
1  1610612738        Boston Celtics          BOS    Celtics       Boston   
2  1610612739   Cleveland Cavaliers          CLE  Cavaliers    Cleveland   
3  1610612740  New Orleans Pelicans          NOP   Pelicans  New Orleans   
4  1610612741         Chicago Bulls          CHI      Bulls      Chicago   

           state  year_founded conference   division  
0        Georgia          1949       East  Southeast  
1  Massachusetts          1946       East   Atlantic  
2           Ohio          1970       East    Central  
3      Louisiana          2002       West  Southwest  
4       Illinois          1966       East    Central  
Enriched team details with conference and division have been written to nba_teams_conference_division.csv


In [9]:
import pandas as pd
from nba_api.stats.static import players

# Fetch list of all active players
all_players = players.get_active_players()

# Convert the list of dictionaries to a DataFrame
active_players_df = pd.DataFrame(all_players)

# Save the DataFrame to a CSV file
active_players_df.to_csv('nba_active_players.csv', index=False)

In [10]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo
import pandas as pd

# Fetch list of all active players
all_players = players.get_active_players()

# Select one player (e.g., the first player in the list)
selected_player = all_players[0]
player_id = selected_player['id']
player_name = selected_player['full_name']

# Fetch player details using the commonplayerinfo endpoint
player_info = commonplayerinfo.CommonPlayerInfo(player_id=player_id)
player_info_df = player_info.get_data_frames()[0]

# Display the player details
print(f"Details for player: {player_name}")
print(player_info_df.head())

# Save the player details to a CSV file
player_info_df.to_csv(f'{player_name}_details.csv', index=False)

Details for player: Precious Achiuwa
   PERSON_ID FIRST_NAME LAST_NAME DISPLAY_FIRST_LAST DISPLAY_LAST_COMMA_FIRST  \
0    1630173   Precious   Achiuwa   Precious Achiuwa        Achiuwa, Precious   

  DISPLAY_FI_LAST       PLAYER_SLUG            BIRTHDATE   SCHOOL  COUNTRY  \
0      P. Achiuwa  precious-achiuwa  1999-09-19T00:00:00  Memphis  Nigeria   

   ...        PLAYERCODE FROM_YEAR TO_YEAR  DLEAGUE_FLAG NBA_FLAG  \
0  ...  precious_achiuwa      2020    2023             N        Y   

  GAMES_PLAYED_FLAG DRAFT_YEAR DRAFT_ROUND  DRAFT_NUMBER GREATEST_75_FLAG  
0                 Y       2020           1            20                N  

[1 rows x 33 columns]
